## Unpacking Elements from Iterables of Arbitrary Length

Python **'star expressions'** can be used to address this problem (Python 3)


In [10]:
user_record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = user_record

print ("Name: ", name)
print ("Email: ", email)
print ("Phone Number: ", phone_numbers)

Name:  Dave
Email:  dave@example.com
Phone Number:  ['773-555-1212', '847-555-1212']


<hr>
#### The starred variable can also be the first one in the list.

In [20]:
*trailing, current = [10, 8, 7, 1, 9, 5, 10, 3]
print (trailing)
print (current)

[10, 8, 7, 1, 9, 5, 10]
3


#### It is worth noting that the star syntax can be especially useful when iterating over a sequence of tuples of varying length.

In [12]:

records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]
def do_foo(x, y):
    print('foo', x, y)
def do_bar(s):
    print('bar', s)
for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


<hr>
#### Star unpacking can also be useful when combined with certain kinds of string processing operations, such as splitting.

In [22]:

line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
print (uname)
print (homedir)
print (sh)


nobody
/var/empty
/usr/bin/false


<hr>
### Keeping the Last N Items
#### You want to keep a limited history of the last few items seen during iteration or during some other kind of processing.

Keeping a limited history is a perfect use for a collections.deque. Using deque(maxlen=N) creates a fixed-sized queue. When new items are added and
the queue is full, the oldest item is automatically removed.

In [5]:
from collections import deque

q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
print ('New Deque of maximum length created: ')
print (q)

q.append(4)
print ('Added item to right and the leftmost item is automatically popped: ')
print (q)

q.appendleft(5)
print ('Added item to left and the rightmost item is automatically popped: ')
print (q)

New Deque of maximum length created: 
deque([1, 2, 3], maxlen=3)
Added item to right and the leftmost item is automatically popped: 
deque([2, 3, 4], maxlen=3)
Added item to left and the rightmost item is automatically popped: 
deque([5, 2, 3], maxlen=3)


Although you could manually perform such operations on a list (e.g., appending, deleting,
etc.), the queue solution is far more elegant and runs a lot faster. Adding or popping items from either end of a queue has O(1) complexity. This is unlike
a list where inserting or removing items from the front of the list is O(N).

<hr>
### Finding Largest or Smallest N items
#### You want to make a list of the largest or smallest N items in a collection.

The heapq module has two functions—nlargest() and nsmallest()—that do exactly
what you want.


In [6]:
import heapq

nums = [1, 6, 3, 5, 33, -34, 45, 66, -87, 0, 21]

print (heapq.nlargest(3, nums))

print (heapq.nsmallest(3, nums))


[66, 45, 33]
[-87, -34, 0]


Both functions also accept a key parameter that allows them to be used with more complicated data structures.

In [12]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
print (cheap)

max_share = heapq.nlargest(1, portfolio, key=lambda s: s['shares'])
print (max_share)

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
[{'name': 'FB', 'shares': 200, 'price': 21.09}]


If you are looking for the N smallest or largest items and N is small compared to the overall size of the collection, these functions provide superior performance. Underneath the covers, they work by first converting the data into a list where items are ordered as a heap.

In [18]:
import heapq

nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
heap = list(nums)

heapq.heapify(heap)
print (heap)

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]


The most important feature of a heap is that heap[0] is always the smallest item. Moreover,
subsequent items can be easily found using the heapq.heappop() method, which
pops off the first item and replaces it with the next smallest item (an operation that
requires O(log N) operations where N is the size of the heap).

In [19]:
print (heapq.heappop(heap))
print (heap)

print (heapq.heappop(heap))
print (heap)

-4
[1, 2, 2, 23, 7, 8, 18, 23, 42, 37]
1
[2, 2, 8, 23, 7, 37, 18, 23, 42]


The **nlargest()** and **nsmallest()** functions are most appropriate if you are trying to find a relatively small number of items. 

If you are simply trying to find the single smallest or largest item (N=1), it is faster to use min() and max(). 

Similarly, if N is about the same size as the collection itself, it is usually faster to sort it first and take a slice (i.e.,
use sorted(items)[:N] or sorted(items)[-N:]).

It should be noted that the actual
implementation of nlargest() and nsmallest() is adaptive in how it operates and will
carry out some of these optimizations on your behalf (e.g., using sorting if N is close to
the same size as the input).

<hr>
### Implementing a Priority Queue
#### Implement a queue that sorts items by a given priority and always returns the item with the highest priority on each pop operation.



In [9]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
        
    def pop(self):
        return heapq.heappop(self._queue)[-1]
    

class Item:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return 'Item(%s)' % self.name
    
q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)

print (q.pop())
print (q.pop())
print (q.pop())
print (q.pop())

Item(bar)
Item(spam)
Item(foo)
Item(grok)


The queue consists of tuples of the form (-priority, index, item). The
priority value is negated to get the queue to sort items from highest priority to lowest
priority. This is opposite of the normal heap ordering, which sorts from lowest to highest
value.
The role of the index variable is to properly order items with the same priority level.
By keeping a constantly increasing index, the items will be sorted according to the order
in which they were inserted. However, the index also serves an important role in making
the comparison operations work for items that have the same priority level.


<hr>
### Mapping Keys to Multiple Values in a Dictionary

#### You want to make a dictionary that maps keys to more than one value (a so-called “multidict”).

Dictionary is a mapping where each key is mapped to a single value. If you want to
map keys to multiple values, you need to store the multiple values in another container
such as a list or set.

To easily construct such dictionaries, you can use defaultdict in the collections
module. A feature of defaultdict is that it automatically initializes the first value so
you can simply focus on adding items.

In [11]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['b'].append(3)
d['a'].append(5)

print (d)


city_list = [('TX','Austin'), ('TX','Houston'),
             ('NY','Albany'), ('NY', 'Syracuse'),
             ('NY', 'Buffalo'), ('NY', 'Rochester'),
             ('TX', 'Dallas'), ('CA','Sacramento'),
             ('CA', 'Palo Alto'), ('GA', 'Atlanta')]


cities_by_state = defaultdict(list)
for state, city in city_list:
    cities_by_state[state].append(city)

print (cities_by_state)


defaultdict(<class 'list'>, {'a': [1, 5], 'b': [3]})
defaultdict(<class 'list'>, {'TX': ['Austin', 'Houston', 'Dallas'], 'NY': ['Albany', 'Syracuse', 'Buffalo', 'Rochester'], 'CA': ['Sacramento', 'Palo Alto'], 'GA': ['Atlanta']})


This recipe is strongly related to the problem of grouping records together in data processing
problems.

<hr>
### Keeping Dictionaries in Order

#### You want to create a dictionary, and you also want to control the order of items when iterating or serializing.

To control the order of items in a dictionary, you can use an OrderedDict from the
collections module. It exactly preserves the original insertion order of data when
iterating.


In [12]:
from collections import OrderedDict

d = OrderedDict()
d['a'] = 1
d['b'] = 2
d['c'] = 3
d['d'] = 4

for key in d:
    print (key, d[key])


a 1
b 2
c 3
d 4


An OrderedDict can be particularly useful when you want to build a mapping that you
may want to later serialize or encode into a different format. For example, if you want
to precisely control the order of fields appearing in a JSON encoding, first building the
data in an OrderedDict will do the trick

In [15]:
import json

print (json.dumps(d))

{"a": 1, "b": 2, "c": 3, "d": 4}


<hr>
### Calculating with dictionaries

#### You want to perform various calculations (e.g., minimum value, maximum value, sorting, etc.) on a dictionary of data.

In order to perform useful calculations on the dictionary contents, it is often useful to
invert the keys and values of the dictionary using zip(). For example, here is how to
find the minimum and maximum price and stock name:

In [25]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

min_price = min(zip(prices.values(), prices.keys()))
print ('Minimum price: ', min_price)

max_price = max(zip(prices.values(), prices.keys()))
print ('Maximum price: ', max_price)

prices_sorted = sorted(zip(prices.values(), prices.keys()))
print ('Prices sorted in desc order', prices_sorted)


min(prices, key=lambda k:prices[k])
prices[min(prices, key=lambda k:prices[k])]

Minimum price:  (10.75, 'FB')
Maximum price:  (612.78, 'AAPL')
Prices sorted in desc order [(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


10.75

The solution involving zip() solves the problem by “inverting” the dictionary into a
sequence of (value, key) pairs. When performing comparisons on such tuples, the
value element is compared first, followed by the key. This gives you exactly the behavior
that you want and allows reductions and sorting to be easily performed on the dictionary
contents using a single statement.

It should be noted that in calculations involving (value, key) pairs, the key will be
used to determine the result in instances where multiple entries happen to have the same
value. For instance, in calculations such as min() and max(), the entry with the smallest
or largest key will be returned if there happen to be duplicate values. For example

In [26]:
prices = { 'AAA' : 45.23, 'ZZZ': 45.23 }
print (min(zip(prices.values(), prices.keys())))
print (max(zip(prices.values(), prices.keys())))

(45.23, 'AAA')
(45.23, 'ZZZ')


### Finding common between two dictionaries

#### You have two dictionaries and want to find out what they might have in common (same keys, same values, etc.).


In [ ]:
a = {
    'x': 1,
    'y': 2,
    'z': 3
}

b = {
    'w': 10,
    'x': 11,
    'y': 2
}

print ('Find keys in common: ')
print (a.keys() & b.keys())

print ('\nFind keys in a that are not in b: ')
print (a.keys() - b.keys())

print ('\nFind (key, value) pairs in common')
print (a.items() & b.items())

print ('\nMake a new dictionary with certain keys removed')
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
print (c)

A dictionary is a mapping between a set of keys and values. The keys() method of a
dictionary returns a keys-view object that exposes the keys. A little-known feature of
keys views is that they also support common set operations such as unions, intersections,
and differences. Thus, if you need to perform common set operations with dictionary
keys, you can often just use the keys-view objects directly without first converting them
into a set.

The items() method of a dictionary returns an items-view object consisting of (key,
value) pairs. This object supports similar set operations and can be used to perform
operations such as finding out which key-value pairs two dictionaries have in common.

Although similar, the values() method of a dictionary does not support the set operations
described in this recipe. In part, this is due to the fact that unlike keys, the items
contained in a values view aren’t guaranteed to be unique. This alone makes certain set
operations of questionable utility. However, if you must perform such calculations, they
can be accomplished by simply converting the values to a set first.

### Removing duplicates while maintaining order from a sequence

If the values in the sequence are hashable, the problem can be easily solved using a set
and a generator.

In [38]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)
            
a = [1, 5, 6, 2, 1, 9, 2, 5, 10]

print (list(dedupe(a)))


[1, 5, 6, 2, 9, 10]


If you are trying to eliminate
duplicates in a sequence of unhashable types (such as dicts), you can make a slight
change to this recipe, as follows:

In [40]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)
            

a = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]

print (list(dedupe(a, key=lambda d: (d['x'], d['y']))))

print (list(dedupe(a, key=lambda d: (d['x']))))


[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]
[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]
